In [ ]:
import sys
import os

REPO_PATH = os.getenv('REPO_PATH')
sys.path.insert(0, os.path.join(REPO_PATH, "py_notebooks/"))
figFolder = os.path.join(REPO_PATH, "py_notebooks/figures/")

from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import plotting

import pandas as pd
import numpy as np
import json
import math

import myunits
u = myunits.units()

import plotly
import plotly.graph_objs as go 
plotly.offline.init_notebook_mode()

cm_to_in = 0.393701
single_col = 8.3
double_col = 17.1
GRAPH_CONFIG = {'showLink': False, 'displaylogo': False,                         
                'modeBarButtonsToRemove':['sendDataToCloud']}

from subprocess import call

In [ ]:
folderNm = os.path.join(REPO_PATH, "data/jl_out/")
#with open(folderNm + "ne_storage_sens_json_data") as data_file:
with open(folderNm + "storage_sens_json_data") as data_file:
    data = json.load(data_file)
# hot_lst = np.linspace(0., 1100., 30) 
# cold_lst = np.linspace(0., 120.e3, 30)
hot_lst = np.linspace(0., 1100., 30) 
cold_lst = np.linspace(0., 120.e3, 30)


In [ ]:
data.keys()

In [ ]:
len(data['bill'].keys())

# Figure 7

In [ ]:
save = False

u['gal_per_ton'] = 10e6/90.e3
u['gal_per_mmbtu'] = 2e6/600.

# x-y values
hot_lst_gal = [el*u['gal_per_mmbtu'] for el in hot_lst]
cold_lst_gal = [el*u['gal_per_ton'] for el in cold_lst]
hot_lst_TJ = [el*u['GJ_per_mmbtu']/1e3 for el in hot_lst]
cold_lst_TJ = [el*u['GJ_per_ton']/1e3 for el in cold_lst]

# 1.32 kWhe -> 1 tonhr # HRC
# 0.45 kWhe -> 1 tonhr # Chiller
# 40 mmbtu/2500 tonhr #HRC
# 1.32*2500/50 kWhe -> 1 mmbtu # HRC
# 50% of cooling loads and 89% of heating loads with the HRCs - but gas heater uses almost no gas
# Assume battery has a one-way efficiency of 94%
u['kWheeq_per_tonhr'] = (1.32+0.45)/2/0.87
u['kWheeq_per_mmbtu'] = 1.32*2500/50/0.87



hot_lst_kWh = [el*u['kWheeq_per_mmbtu'] for el in hot_lst]
cold_lst_kWh = [el*u['kWheeq_per_tonhr'] for el in cold_lst]

# z values
peaks = [[el/1e3 for el in ll] for ll in data["peak"]]

reductions = [[(data["agg_dollar"][0][0]-el)/1E3 for el in ll] for ll in data["agg_dollar"]]

vmax=30.

reductions_normed = []
kWheeq_list = []
for icold, ll in enumerate(data["agg_dollar"]):
    curr_r =[]
    curr_k =[]
    for ihot, el in enumerate(ll):
#        kWheeq = hot_lst_kWh[ihot] + cold_lst_kWh[icold]
        kWheeq = max(cold_lst_kWh[icold],hot_lst_kWh[ihot])
        if kWheeq == 0.:
            red = 0.
        else:
            red = min(vmax,(data["agg_dollar"][0][0]-el)/kWheeq)
        curr_r.append(red)
        curr_k.append(kWheeq/1e3)
    reductions_normed.append(curr_r)
    kWheeq_list.append(curr_k)

    
# fill with None
chillers = data["chillers"].copy()
for ii, el in enumerate(data["chillers"]):
    for jj, _ in enumerate(el):
        if data["chillers"][ii][jj] > 0.:
            peaks[ii][jj] = None
            reductions[ii][jj] = None
            reductions_normed[ii][jj] = None
            chillers[ii][jj] = 1.

contour_dict = contours=dict(coloring='lines',
                             showlabels = True,
                             labelfont = dict(
                                 family = 'Raleway',
                                 size = 16,
                                 color = 'black')
                            )
marker_dict = dict(size = 15,
                   color = 'rgb(255, 255, 0)',
                   line = dict(width = 2)
                  )

colorscale = [[0, 'rgb(0, 0, 0)'],
        [0.33, 'rgb(0, 0, 0)'],

        [0.33, 'rgb(60, 60, 60)'],
        [0.66, 'rgb(60, 60, 60)'],

        [0.66, 'rgb(140, 140, 140)'],
        [1., 'rgb(140, 140, 140)']]
        
#val_contour
def ch_contour():
    return go.Contour(z=chillers,
                       x=hot_lst_TJ,
                       y=cold_lst_TJ,
                       line=dict(width=3., dash='dash'),
                        contours=dict(
                            coloring='lines',
                            start=0,
                            end=1,
                            size=1,
                        ),
                        colorscale=colorscale,
                        autocontour=False,
                        showscale= False
                       )
def txt_scatter():
    return go.Scatter(x=[0.633], y=[1.134],
                         showlegend=False,
                         marker = marker_dict,
                         mode = 'markers+text',
                         text = ['Stanford'],
                         textposition = 'bottom center'
                     )
    
plot_data = [go.Contour(z=peaks,
                                x=hot_lst_TJ,
                                y=cold_lst_TJ,
                                colorbar=dict(title='$',x=0.41, len=0.4, y=0.78),
                                xaxis='x',
                                yaxis='y',
                                contours=contour_dict,
                                showscale=False
                               ),
                     go.Contour(z=reductions,
                                x=hot_lst_TJ,
                                y=cold_lst_TJ,
                                #colorbar=dict(title='$',x=0.99, len=0.4, y=0.78),
                                xaxis='x2',
                                yaxis='y2',
                                contours=contour_dict,
                                showscale=False
                               ),
                     go.Contour(z=kWheeq_list,
                                x=hot_lst_TJ,
                                y=cold_lst_TJ,
                                #colorbar=dict(title='$',x=0.41, len=0.4, y=0.2),
                                xaxis='x3',
                                yaxis='y3',
                                contours=contour_dict,
                                showscale=False
                               ),
                     go.Contour(z=reductions_normed,
                                x=hot_lst_TJ,
                                y=cold_lst_TJ,
                                #colorbar=dict(title='$',x=0.99, len=0.4, y=0.2),
                                xaxis='x4',
                                yaxis='y4',
                                contours=contour_dict,
                                showscale=False
                               ),
                    ]

for x,y in [('x','y'), ('x2','y2'), ('x3','y3'), ('x4','y4')]:
    sc = txt_scatter()
    sc['xaxis'], sc['yaxis'] = x, y
    plot_data += [sc]

for x,y in [('x','y'), ('x2','y2'), ('x4','y4')]:
    cr = ch_contour()
    cr['xaxis'], cr['yaxis'] = x,y
    plot_data += [cr]
    ch_contour

ch_ann = {'font': {'size': 22},
                       'showarrow': False,
                       'xanchor': 'center',
                       'xref': 'x',
                       'y': -0.08,
                       'yanchor': 'bottom',
                       'yref': 'y',
                      'bgcolor':'white',
                      'bordercolor':'#c7c7c7'}

tit_ann = {'font': {'size': 22},
           'showarrow': False,
           'xanchor': 'center',
           'xref': 'paper',
           'yanchor': 'bottom',
           'yref': 'paper'}

tit_ann1 = tit_ann.copy()
tit_ann1['text'], tit_ann1['x'], tit_ann1['y'] = '(a) Annual Peak Load (MW)', 0.225, 1.

tit_ann2 = tit_ann.copy()
tit_ann2['text'], tit_ann2['x'], tit_ann2['y'] = '(b) Operating Cost Savings (k$/year)', 0.775, 1.

tit_ann3 = tit_ann.copy()
tit_ann3['text'], tit_ann3['x'], tit_ann3['y'] = '(c) Equivalent electrochemical storage (MWhe-eq)', 0.225, .45

tit_ann4 = tit_ann.copy()
tit_ann4['text'], tit_ann4['x'], tit_ann4['y'] = '(d) Normalized Operating Cost Savings ($/kWhe-eq/year)', 0.775, .45

layout = dict(
    title='',
    font=dict(size=18),
    autosize=False,
    width=2*650,
    height=2*500,
    xaxis=dict(title="", domain=[0., .45]),
    xaxis2=dict(title="", domain=[0.55, 1.]),
    xaxis3=dict(title="Hot storage (TJ)", domain=[0., 0.45], anchor='y3'),
    xaxis4=dict(title="Hot storage (TJ)", domain=[0.55, 1.], anchor='y4'),
    yaxis=dict(title="Cold storage (TJ)", domain=[0.55, 1.]),
    yaxis2=dict(title="", domain=[0.55, 1.], anchor='x2'),
    yaxis3=dict(title="Cold storage (TJ)", domain=[0., 0.45], anchor='x3'),
    yaxis4=dict(title="", domain=[0., 0.45], anchor='x4'),
    annotations=[tit_ann1, tit_ann2, tit_ann3, tit_ann4]
)

fig = go.Figure(data=plot_data, layout=layout)
plotly.offline.iplot(fig, config=GRAPH_CONFIG)
if save:
    call(['xvfb-run', 'orca', 'graph', json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder),
      '-o', 'fig7.pdf', '--format', 'pdf'])